In [ ]:
pip install pandas folium geopy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install koreanize-matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 55.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dest_name = pd.read_csv('/content/drive/MyDrive/KIKmix_20230701.csv')
dest_name.head()

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자
0,1100000000,서울특별시,NaN,NaN,1100000000,서울특별시,19880423,NaN
1,1111000000,서울특별시,종로구,NaN,1111000000,종로구,19880423,NaN
2,1111051500,서울특별시,종로구,청운효자동,1111010100,청운동,20081101,NaN
3,1111051500,서울특별시,종로구,청운효자동,1111010200,신교동,20081101,NaN
4,1111051500,서울특별시,종로구,청운효자동,1111010300,궁정동,20081101,NaN


In [ ]:
bf_festival_file_path = '/content/drive/MyDrive/bf_od'
festival_file_path = '/content/drive/MyDrive/f_od'

In [ ]:
# MyDrive의 data 폴더에서 모든 CSV 파일 경로 가져오기
bf_festival_csv_files = glob.glob('/content/drive/MyDrive/bf_od/*.csv')
festival_csv_files = glob.glob('/content/drive/MyDrive/f_od/*.csv')

bf_df_list = []  # 각 파일의 데이터프레임을 저장할 리스트
for file in bf_festival_csv_files:
    print(f"파일명: {file.split('/')[-1]}")  # 파일명만 출력
    df = pd.read_csv(file)  # 각 파일의 데이터 읽기
    bf_df_list.append(df)   # 리스트에 데이터프레임 추가

# 모든 데이터프레임을 하나로 합치기
bf_df = pd.concat(bf_df_list, ignore_index=True)

f_df_list = []  # 각 파일의 데이터프레임을 저장할 리스트
for file in festival_csv_files:
    print(f"파일명: {file.split('/')[-1]}")  # 파일명만 출력
    df = pd.read_csv(file)  # 각 파일의 데이터 읽기
    f_df_list.append(df)    # 리스트에 데이터프레임 추가

# 모든 데이터프레임을 하나로 합치기
f_df = pd.concat(f_df_list, ignore_index=True)

파일명: od_20231001_1.csv
파일명: od_20230927_1.csv
파일명: od_20230928_1.csv
파일명: od_20230929_1.csv
파일명: od_20230930_1.csv
파일명: od_20230925_1.csv
파일명: od_20230926_1.csv
파일명: od_20231011_1.csv
파일명: od_20231008_1.csv
파일명: od_20231009_1.csv
파일명: od_20231010_1.csv
파일명: od_20231014_1.csv
파일명: od_20231012_1.csv
파일명: od_20231013_1.csv


In [ ]:
#[법정동(행정동)]
#진주시 성북동(본성동, 남성동, 인사동, 중안동, 계동, 봉곡동)
#진주시 중앙동(동성동, 대안동, 평안동, 수정동, 장대동, 옥봉동)
#진주시 상봉동
#진주시 상평동
#진주시 상대동
#진주시 충무공동
#진주시 가호동(가좌동, 호탄동)
#진주시 이현동
#진주시 신안동
#진주시 평거동
#진주시 초장동(초전동)
#진주시 예하리

target_dest_file_path = '/content/drive/MyDrive/target_dest_code.csv'
target_dest_df = pd.read_csv(target_dest_file_path)
target_dest_df.head()

,dest_hdong_cd,province,city,vilage,dest_bdong_cd,avenue
0,4817055500,경상남도,진주시,성북동,4817010500,본성동
1,4817055500,경상남도,진주시,성북동,4817010700,남성동
2,4817055500,경상남도,진주시,성북동,4817010800,인사동
3,4817055500,경상남도,진주시,성북동,4817011100,중안동
4,4817055500,경상남도,진주시,성북동,4817011200,계동


In [ ]:
target_codes = target_dest_df['dest_hdong_cd'].unique()

# bf_df에서 'dest_hdong_cd'가 target_codes에 있는 데이터 필터링
bf_filtered = bf_df[bf_df['dest_hdong_cd'].isin(target_codes)]

# f_df에서 'dest_hdong_cd'가 target_codes에 있는 데이터 필터링
f_filtered = f_df[f_df['dest_hdong_cd'].isin(target_codes)]

In [ ]:
bf_filtered

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,od_dist_avg,od_duration_avg,od_cnts
365,4817069500,4817056500,20231001,10:00,10:00,0,1,0.0,0.0,4,10325,12,7
1039,4817067800,4817069500,20231001,22:00,23:00,0,2,0.0,0.0,0,15815,28,6
1395,4817056500,4817059500,20231001,22:00,22:00,1,4,0.0,4.0,4,7569,4,8
1514,4817074000,4817059500,20231001,17:00,17:00,1,2,0.0,4.0,0,30043,36,8
2423,4817059500,4817056500,20231001,14:00,14:00,0,1,0.0,0.0,0,2204,12,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24885368,4817067800,4817059500,20230926,09:00,10:00,1,1,0.0,0.0,2,52095,41,5
24886023,4817067300,4817055500,20230926,08:00,08:00,1,4,0.0,0.0,1,27713,29,5
24887094,4817056500,4817055500,20230926,12:00,12:00,0,6,0.0,1.0,1,5373,13,5
24887404,4817069500,4817067300,20230926,20:00,20:00,1,4,0.0,0.0,4,6431,10,5


In [ ]:
bf_filtered_nh = bf_filtered[~bf_filtered['dest_purpose'].isin([0, 2])]
bf_filtered_nh

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,od_dist_avg,od_duration_avg,od_cnts
365,4817069500,4817056500,20231001,10:00,10:00,0,1,0.0,0.0,4,10325,12,7
1395,4817056500,4817059500,20231001,22:00,22:00,1,4,0.0,4.0,4,7569,4,8
3287,4817041000,4817074000,20231001,15:00,16:00,0,2,0.0,0.0,4,45475,55,8
4050,4613080000,4817059500,20231001,08:00,10:00,1,0,0.0,5.0,4,315066,155,16
5915,4817073000,4817055500,20231001,16:00,17:00,1,0,1.0,2.0,3,39987,57,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24883188,4817051500,4817056500,20230926,10:00,12:00,1,3,0.0,4.0,4,55893,94,5
24885074,4817051500,4817055500,20230926,08:00,08:00,1,1,0.0,0.0,4,8026,20,5
24886023,4817067300,4817055500,20230926,08:00,08:00,1,4,0.0,0.0,1,27713,29,5
24887094,4817056500,4817055500,20230926,12:00,12:00,0,6,0.0,1.0,1,5373,13,5


In [ ]:
f_filtered

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,od_dist_avg,od_duration_avg,od_cnts
43,4817067800,4817067300,20231011,10:00,10:00,0,5,4.0,0.0,0,1590,12,8
104,4817055500,4817071000,20231011,17:00,18:00,0,3,0.0,1.0,0,52819,42,7
614,4817025000,4817071000,20231011,19:00,19:00,0,0,0.0,3.0,4,54212,19,10
1305,4817073000,4817071000,20231011,18:00,18:00,1,1,0.0,2.0,0,9883,16,19
2628,4817051500,4817067300,20231011,21:00,21:00,0,1,0.0,0.0,0,2181,7,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24381293,4817069500,4817067300,20231013,11:00,12:00,1,4,0.0,4.0,4,30939,58,5
24381423,4817075000,4817074000,20231013,17:00,17:00,1,2,0.0,4.0,4,22050,25,5
24381699,4817067300,4817071000,20231013,19:00,20:00,1,5,0.0,4.0,0,32796,54,5
24381700,4824025000,4817074000,20231013,17:00,17:00,0,3,0.0,1.0,0,63648,34,5


In [ ]:
f_filtered_nh = f_filtered[~f_filtered['dest_purpose'].isin([0,2])]
f_filtered_nh

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,od_dist_avg,od_duration_avg,od_cnts
614,4817025000,4817071000,20231011,19:00,19:00,0,0,0.0,3.0,4,54212,19,10
2916,4817067300,4817059500,20231011,10:00,11:00,0,4,0.0,4.0,4,34030,64,7
4567,4817025000,4817056500,20231011,15:00,15:00,1,4,0.0,4.0,3,35174,18,7
4684,4817055500,4817056500,20231011,13:00,13:00,1,3,0.0,1.0,1,4087,14,18
5314,4817051500,4817074000,20231011,13:00,13:00,0,2,0.0,0.0,4,21699,25,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24380214,4817068000,4817067300,20231013,15:00,16:00,0,4,0.0,1.0,1,72040,58,5
24381245,4817051500,4817074000,20231013,13:00,13:00,1,2,1.0,0.0,1,18055,26,5
24381293,4817069500,4817067300,20231013,11:00,12:00,1,4,0.0,4.0,4,30939,58,5
24381423,4817075000,4817074000,20231013,17:00,17:00,1,2,0.0,4.0,4,22050,25,5


In [ ]:
# origin_hdong_cd의 값 개수 세기
origin_hdong_counts = bf_filtered_nh['origin_hdong_cd'].value_counts()

# 각 origin_hdong_cd에 맞는 시도명과 시군구명 찾기 및 출력
for code, count in origin_hdong_counts.items():
    # KIKmix에서 행정동코드에 해당하는 시도명/시군구명 찾기
    matching_info = dest_name[dest_name['행정동코드'] == code]

    if not matching_info.empty:
        sido = matching_info.iloc[0]['시도명']
        sigungu = matching_info.iloc[0]['시군구명']
        eupmyeondong = matching_info.iloc[0]['읍면동명']

        print(f"{sido} {sigungu} {eupmyeondong}: {count}개")

경상남도 진주시 천전동: 3975개
경상남도 진주시 성북동: 3132개
경상남도 진주시 충무공동: 2544개
경상남도 진주시 판문동: 2167개
경상남도 진주시 초장동: 2019개
경상남도 진주시 평거동: 1855개
경상남도 진주시 하대동: 1827개
경상남도 진주시 가호동: 1766개
경상남도 진주시 신안동: 1373개
경상남도 진주시 상대동: 1371개
경상남도 진주시 이현동: 1146개
경상남도 진주시 중앙동: 1142개
경상남도 진주시 상평동: 922개
경상남도 진주시 상봉동: 904개
경상남도 진주시 금산면: 851개
경상남도 진주시 정촌면: 519개
경상남도 진주시 내동면: 402개
경상남도 사천시 사천읍: 277개
경상남도 진주시 문산읍: 255개
경상남도 사천시 사남면: 174개
경상남도 진주시 명석면: 174개
경상남도 진주시 집현면: 142개
경상남도 산청군 신안면: 115개
경상남도 사천시 정동면: 93개
경상남도 산청군 단성면: 90개
경상남도 산청군 금서면: 90개
경상남도 사천시 동서동: 73개
경상남도 사천시 벌용동: 70개
경상남도 사천시 곤명면: 64개
경상남도 산청군 산청읍: 63개
경상남도 함양군 함양읍: 56개
경상남도 진주시 미천면: 52개
경상남도 진주시 대곡면: 48개
경상남도 산청군 시천면: 47개
경상남도 하동군 진교면: 47개
경상남도 사천시 용현면: 46개
경상남도 진주시 진성면: 45개
경상남도 사천시 남양동: 43개
경상남도 하동군 북천면: 42개
경상남도 하동군 옥종면: 39개
경상남도 사천시 곤양면: 35개
경상남도 사천시 축동면: 31개
경상남도 진주시 이반성면: 30개
경상남도 거창군 거창읍: 29개
경상남도 하동군 하동읍: 28개
경상남도 사천시 서포면: 28개
경상남도 산청군 신등면: 26개
경상남도 남해군 남해읍: 26개
경상남도 진주시 사봉면: 25개
경상남도 고성군 고성읍: 25개
경상남도 진주시 수곡면: 24개
경상남도 합천군 합천읍: 23개
경상남도 진주시 금곡면: 23개
경상남도 사천시 

In [ ]:
# 축제 전 OD 데이터 분석
origin_hdong_counts = bf_filtered_nh['origin_hdong_cd'].value_counts()

# 행정동 코드에 해당하는 시도명, 시군구명 매칭 후 저장
results = []  # 결과를 저장할 리스트

for code, count in origin_hdong_counts.items():
    matching_info = dest_name[dest_name['행정동코드'] == code]

    if not matching_info.empty:
        sido = matching_info.iloc[0]['시도명']
        sigungu = matching_info.iloc[0]['시군구명']
        results.append({'시도명': sido, '시군구명': sigungu, 'count': count})
    else:
        print(f"코드 {code}: {count}개 (매칭되는 시도명/시군구명 없음)")

df_results = pd.DataFrame(results)

# 출발 데이터를 시단위로 묶기 위해 특별시, 광역시, 특별자치도를 special_cities로 구분하고, 그룹명 생성시 시군구 앞3글자가 동일하면 같은 그룹으로 묶도록 함
# 평택시 -> 3글자, 의정부시 -> 앞 3글자로 그룹화 가능, 용인시 기흥구 -> 앞 3글자로 그룹화 가능
# 특별시/광역시/특별자치도

special_cities = [
    '서울특별시', '부산광역시', '대구광역시', '대전광역시',
    '인천광역시', '광주광역시', '울산광역시', '제주특별자치도'
]

# 그룹명 생성: special_cities는 시도명 기준으로, 나머지는 시도명 + 시군구 앞 3글자
def generate_group_name(row):
    if row['시도명'] in special_cities:
        return row['시도명']  # 특별시/광역시는 시도명으로 그룹화
    else:
        # 시군구명이 결측값일 경우 기본값으로 처리
        sigungu = row['시군구명'] if pd.notna(row['시군구명']) else ""
        return f"{row['시도명']} {sigungu[:3]}"  # 시군구명 앞 3글자 포함

# 그룹명 생성 및 추가
df_results['그룹명'] = df_results.apply(generate_group_name, axis=1)

# 그룹화하여 개수 합산
grouped_df = df_results.groupby('그룹명', as_index=False).sum()

# 개수 기준으로 내림차순 정렬
grouped_df = grouped_df.sort_values(by='count', ascending=False)

# 결과 출력
for idx, row in grouped_df.iterrows():
    print(f"{row['그룹명']}: {row['count']}개")

코드 4139065000: 1개 (매칭되는 시도명/시군구명 없음)
경상남도 진주시: 28789개
경상남도 사천시: 992개
경상남도 산청군: 482개
경상남도 창원시: 292개
경상남도 하동군: 238개
부산광역시: 222개
경상남도 남해군: 105개
경상남도 김해시: 104개
경상남도 고성군: 103개
대구광역시: 103개
경상남도 거제시: 100개
경상남도 합천군: 88개
경상남도 함양군: 86개
경상남도 통영시: 73개
경상남도 의령군: 57개
대전광역시: 55개
울산광역시: 54개
서울특별시: 52개
경상남도 함안군: 48개
전라남도 광양시: 40개
경상남도 거창군: 40개
전라남도 순천시: 37개
경상남도 양산시: 32개
전라남도 여수시: 24개
광주광역시: 23개
경상북도 구미시: 21개
전라북도 전주시: 19개
경상남도 밀양시: 18개
경상북도 경주시: 15개
전라남도 구례군: 14개
세종특별자치시 : 13개
충청북도 청주시: 13개
경상북도 경산시: 12개
경기도 화성시: 11개
충청남도 천안시: 11개
경상북도 포항시: 11개
충청남도 아산시: 10개
전라북도 남원시: 9개
경기도 성남시: 8개
경기도 용인시: 8개
충청남도 금산군: 7개
인천광역시: 7개
경기도 평택시: 7개
경상남도 창녕군: 6개
경상북도 칠곡군: 5개
경상북도 안동시: 5개
경상북도 영천시: 5개
제주특별자치도: 5개
경상북도 고령군: 5개
전라북도 무주군: 5개
충청남도 예산군: 5개
경상북도 성주군: 4개
전라남도 고흥군: 4개
경기도 수원시: 4개
경기도 시흥시: 4개
전라북도 장수군: 4개
전라남도 해남군: 4개
전라북도 임실군: 4개
충청남도 당진시: 4개
충청북도 진천군: 4개
경상북도 의성군: 4개
경상북도 문경시: 4개
충청남도 서천군: 3개
경상북도 김천시: 3개
경상북도 상주시: 3개
전라남도 완도군: 3개
전라북도 고창군: 3개
경기도 이천시: 3개
경기도 안성시: 3개
전라북도 김제시: 3개
전라남도 목포시: 3개
전라남도 담양군: 3개
전라북도 익산

In [ ]:
# 축제 기간 OD 데이터 분석
origin_hdong_counts = f_filtered_nh['origin_hdong_cd'].value_counts()

# 행정동 코드에 해당하는 시도명, 시군구명 매칭 후 저장
results = []  # 결과를 저장할 리스트

for code, count in origin_hdong_counts.items():
    matching_info = dest_name[dest_name['행정동코드'] == code]

    if not matching_info.empty:
        sido = matching_info.iloc[0]['시도명']
        sigungu = matching_info.iloc[0]['시군구명']
        results.append({'시도명': sido, '시군구명': sigungu, 'count': count})
    else:
        print(f"코드 {code}: {count}개 (매칭되는 시도명/시군구명 없음)")

df_results = pd.DataFrame(results)

# 특별시/광역시/특별자치도 목록 정의
special_cities = [
    '서울특별시', '부산광역시', '대구광역시', '대전광역시',
    '인천광역시', '광주광역시', '울산광역시', '제주특별자치도'
]

# 그룹명 생성: special_cities는 시도명 기준으로, 나머지는 시도명 + 시군구 앞 3글자
def generate_group_name(row):
    if row['시도명'] in special_cities:
        return row['시도명']  # 특별시/광역시는 시도명으로 그룹화
    else:
        # 시군구명이 결측값일 경우 기본값으로 처리
        sigungu = row['시군구명'] if pd.notna(row['시군구명']) else ""
        return f"{row['시도명']} {sigungu[:3]}"  # 시군구명 앞 3글자 포함

# 그룹명 생성 및 추가
df_results['그룹명'] = df_results.apply(generate_group_name, axis=1)

# 그룹화하여 개수 합산
grouped_df = df_results.groupby('그룹명', as_index=False).sum()

# 개수 기준으로 내림차순 정렬
grouped_df = grouped_df.sort_values(by='count', ascending=False)

# 결과 출력
for idx, row in grouped_df.iterrows():
    print(f"{row['그룹명']}: {row['count']}개")

경상남도 진주시: 31650개
경상남도 사천시: 946개
경상남도 창원시: 485개
경상남도 산청군: 313개
부산광역시: 307개
경상남도 하동군: 157개
경상남도 김해시: 155개
대구광역시: 155개
경상남도 거제시: 106개
경상남도 남해군: 97개
경상남도 통영시: 90개
경상남도 합천군: 86개
경상남도 고성군: 84개
경상남도 함양군: 70개
전라남도 순천시: 69개
경상남도 의령군: 68개
경상남도 함안군: 58개
울산광역시: 58개
전라남도 광양시: 56개
광주광역시: 48개
서울특별시: 43개
전라남도 여수시: 42개
경상남도 양산시: 37개
전라북도 전주시: 37개
경상남도 거창군: 32개
대전광역시: 25개
경상북도 구미시: 21개
충청북도 청주시: 19개
경상북도 경주시: 17개
경상남도 창녕군: 13개
경상북도 포항시: 12개
경상남도 밀양시: 11개
경상북도 경산시: 10개
경기도 수원시: 8개
전라남도 나주시: 7개
충청남도 천안시: 7개
전라북도 장수군: 7개
충청북도 제천시: 6개
전라남도 목포시: 6개
경기도 용인시: 6개
전라북도 남원시: 6개
경상북도 성주군: 5개
세종특별자치시 : 5개
경상북도 칠곡군: 5개
전라북도 군산시: 4개
전라북도 익산시: 4개
경기도 평택시: 4개
충청남도 금산군: 4개
경상북도 김천시: 4개
경상북도 안동시: 4개
충청남도 서천군: 3개
충청남도 아산시: 3개
전라남도 곡성군: 3개
인천광역시: 3개
충청남도 부여군: 3개
경기도 성남시: 3개
전라북도 순창군: 3개
전라북도 무주군: 3개
경기도 화성시: 3개
경상북도 영천시: 3개
충청북도 진천군: 3개
전라북도 진안군: 3개
경상북도 청도군: 2개
경상북도 고령군: 2개
전라남도 구례군: 2개
전라남도 강진군: 2개
전라남도 화순군: 2개
전라남도 해남군: 2개
제주특별자치도: 2개
강원특별자치도 태백시: 2개
경기도 양주시: 2개
경기도 이천시: 2개
경기도 안산시: 2개
전라남도 장흥군: 2개
전라남도 무안군: 2개
경상북도 영덕군

In [ ]:
# 행정동 코드 -> (시도명, 시군구명 앞 3글자) 매핑 함수 생성
def get_region_info(hdong_code):
    matching_info = dest_name[dest_name['행정동코드'] == hdong_code]

    if not matching_info.empty:
        # 시도명 추출
        sido = matching_info.iloc[0]['시도명']

        # 시군구명 추출 및 결측값 처리
        sigungu = matching_info.iloc[0]['시군구명']
        if pd.isna(sigungu):
            sigungu_prefix = ""  # 결측값인 경우 빈 문자열 사용
        else:
            sigungu_prefix = sigungu[:3]  # 앞 3글자 추출

        return f"{sido} {sigungu_prefix}"
    else:
        return None  # 매칭되는 정보가 없으면 None 반환

# bf_filtered_nh의 origin_hdong_cd를 시도명+시군구명 앞 3글자로 변환
bf_filtered_nh['region'] = bf_filtered_nh['origin_hdong_cd'].apply(get_region_info)

# f_filtered_nh의 origin_hdong_cd를 시도명+시군구명 앞 3글자로 변환
f_filtered_nh['region'] = f_filtered_nh['origin_hdong_cd'].apply(get_region_info)

# 각 데이터프레임에서 고유한 region 값 집합 생성
bf_regions = set(bf_filtered_nh['region'].dropna())
f_regions = set(f_filtered_nh['region'].dropna())

# 두 집합의 차이 계산
only_in_bf = bf_regions - f_regions  # bf_filtered_nh에만 있는 지역
only_in_f = f_regions - bf_regions  # f_filtered_nh에만 있는 지역
common_regions = bf_regions & f_regions  # 공통으로 포함된 지역

# 결과 출력
print("bf_filtered_nh에만 있는 지역:")
for region in only_in_bf:
    print(region)

print("\nf_filtered_nh에만 있는 지역:")
for region in only_in_f:
    print(region)

print(f"\n공통으로 포함된 지역 개수: {len(common_regions)}")

bf_filtered_nh에만 있는 지역:
충청남도 보령시
제주특별자치도 서귀포
충청남도 서산시
인천광역시 연수구
강원특별자치도 춘천시
충청남도 청양군
서울특별시 성동구
인천광역시 남동구
강원특별자치도 횡성군
경기도 안성시
경상북도 영주시
서울특별시 성북구
충청북도 괴산군
경상북도 의성군
경기도 여주시
전라북도 부안군
전라남도 신안군
전라남도 영암군
경기도 의정부
충청남도 태안군
충청남도 당진시
경상북도 상주시
전라북도 고창군
서울특별시 동대문
경기도 시흥시
경기도 의왕시
충청북도 옥천군
서울특별시 도봉구
전라남도 장성군
전라북도 완주군
전라남도 담양군
경기도 안양시
충청남도 예산군
강원특별자치도 삼척시
충청북도 영동군
충청북도 음성군
전라북도 김제시

f_filtered_nh에만 있는 지역:
전라남도 나주시
강원특별자치도 태백시
충청북도 단양군
전라남도 화순군
경상북도 영양군
서울특별시 종로구
경기도 포천시
서울특별시 강북구
서울특별시 은평구
전라남도 곡성군
충청남도 홍성군
인천광역시 미추홀
서울특별시 금천구
충청북도 충주시
충청북도 보은군
전라남도 강진군
경기도 안산시
서울특별시 강동구

공통으로 포함된 지역 개수: 137


In [ ]:
# 1. 각 데이터프레임에서 고유한 region 값 집합 생성
bf_regions = set(bf_filtered_nh['region'].dropna())
f_regions = set(f_filtered_nh['region'].dropna())

# 2. 공통으로 포함된 지역 계산
common_regions = bf_regions & f_regions  # 공통 지역

# 3. 공통 지역별 데이터 수 계산
bf_counts = bf_filtered_nh.groupby('region').size()  # bf_filtered_nh 지역별 개수
f_counts = f_filtered_nh.groupby('region').size()  # f_filtered_nh 지역별 개수

# 4. 공통 지역에서의 데이터 수 증감 계산
changes = []

for region in common_regions:
    bf_count = bf_counts.get(region, 0)  # bf_filtered_nh의 데이터 수 (없으면 0)
    f_count = f_counts.get(region, 0)  # f_filtered_nh의 데이터 수 (없으면 0)
    change = f_count - bf_count  # 데이터 수 증감 계산
    changes.append({'region': region, 'bf_count': bf_count, 'f_count': f_count, 'change': change})

# 5. 결과를 DataFrame으로 정리
changes_df = pd.DataFrame(changes)

# 6. 증감 결과 출력
print("공통 지역별 데이터 수 증감:")
print(changes_df)

# 7. 증감이 큰 순서로 정렬하여 출력 (옵션)
changes_df_sorted = changes_df.sort_values(by='change', ascending=False)
print("\n증감이 큰 순서로 정렬된 데이터:")
print(changes_df_sorted)

공통 지역별 데이터 수 증감:
       region  bf_count  f_count  change
0    부산광역시 서구         9        9       0
1     경기도 성남시         8        3      -5
2    대구광역시 서구         2        5       3
3    전라남도 구례군        14        2     -12
4    경상남도 산청군       482      313    -169
..        ...       ...      ...     ...
132  경상남도 거제시       100      106       6
133   경기도 광주시         2        1      -1
134  충청북도 제천시         1        6       5
135  경상남도 통영시        73       90      17
136  광주광역시 동구         1        3       2

[137 rows x 4 columns]

증감이 큰 순서로 정렬된 데이터:
        region  bf_count  f_count  change
92    경상남도 진주시     28789    31650    2861
54    경상남도 창원시       292      485     193
100   경상남도 김해시       104      155      51
76    전라남도 순천시        37       69      32
53   부산광역시 강서구        12       34      22
..         ...       ...      ...     ...
125   경상남도 함양군        86       70     -16
127   경상남도 고성군       103       84     -19
69    경상남도 사천시       992      946     -46
73    경상남도 하동군       238   